<a href="https://colab.research.google.com/github/foxtrotmike/CS909/blob/master/autodiff_forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **How to Build an Automatic Differentiation Library: Understanding Dual Numbers and forward differentiation**
(Fayyaz Minhas)

Automatic differentiation (autodiff) is a powerful technique that allows us to compute derivatives of functions efficiently. One of the simplest ways to understand autodiff is through **dual numbers**.

This tutorial will introduce:
1. **The concept of dual numbers**
2. **How dual numbers can be implemented in Python**
3. **How differentiation arises naturally from dual numbers**

We will focus on **basic operations**: **addition, multiplication, and power**, and show how differentiation emerges from dual number arithmetic.


## **1️⃣ Understanding Dual Numbers**

### **What are Dual Numbers?**  

Dual numbers extend real numbers and are written as:

$$
x + \epsilon y
$$

where:
- $x$ is the **real part** (the function value),
- $y$ is the **dual part** (the derivative),
- $\epsilon$ is an infinitesimal number with the special (and essential) property:

  $$
  \epsilon^2 = 0
  $$

This means that **higher-order powers of $\epsilon$ vanish**, making dual numbers ideal for encoding derivatives as the Taylor series of $ f(x) $ around $ x $ is:

$$
f(x + \epsilon) = f(x) + \epsilon f'(x) + \frac{\epsilon^2}{2} f''(x) + \mathcal{O}(\epsilon^3)
$$

Since **$ \epsilon^2 = 0 $** (by definition), all higher-order terms vanish, leaving us with:

$$
f(x + \epsilon) = f(x) + \epsilon f'(x)
$$

This equation shows that **evaluating a function with a dual number automatically computes both the function's value and its derivative**.

This means that when we plug a **dual number** $ x + \epsilon $ into a function, the result will **naturally contain both**:
1. The function value $ f(x) $ in the **real part**.
2. The function’s derivative $ f'(x) $ in the **dual part**.

---

## **Example:**
Consider the function:

$$
f(x) = x^2
$$

If we evaluate this function at $ x = 3 $ using **dual numbers**, we set:

$$
x = 3 + \epsilon
$$

Now, compute:

$$
f(x + \epsilon) = (3 + \epsilon)^2
$$

Expanding using algebra:

$$
= 9 + 6\epsilon + \epsilon^2
$$

Since $ \epsilon^2 = 0 $, this simplifies to:

$$
f(3 + \epsilon) = 9 + 6\epsilon
$$

Thus:
- **Real part** $ 9 $ is just $ f(3) $.
- **Dual part** $ 6 $ is the derivative $ f'(3) = 2(3) = 6 $.

✅ **By evaluating the function at $ x + \epsilon $, we automatically get both $ f(x) $ and $ f'(x) $!**

---

##**Why This Works for Any Function**
The key idea here is that **each function operation correctly propagates derivatives**.  

If $ f(x) $ is made up of simpler functions (addition, multiplication, power, etc.), then using **dual numbers** ensures that every step follows **differentiation rules naturally**.

For example, if we apply the product rule to:

$$
g(x) = x^3 + 2x
$$

Using dual numbers:

$$
g(4 + \epsilon) = (4 + \epsilon)^3 + 2(4 + \epsilon)
$$

Expanding:

$$
= 64 + 3(4^2) \epsilon + 8 + 2\epsilon
$$

$$
= 72 + 50\epsilon
$$

So, **the real part is** $ g(4) = 72 $, **and the dual part is** $ g'(4) = 50 $.

---

## **🔹 Key Takeaways**
✅ **Evaluating a function at $ x + \epsilon $ automatically computes both the function's value and its derivative.**  
✅ **This works because of the Taylor expansion, where $ \epsilon^2 = 0 $ simplifies everything.**  
✅ **Each mathematical operation correctly propagates derivatives, ensuring we get the correct derivative for any function.**

---



## **2️⃣ Implementing Dual Numbers in Python**

In order to implement, these magical numbers, We will define a **`DualNumber`** class that supports different functions such as:
- **Addition**
- **Multiplication**
- **Exponentiation (Power function)**


In [1]:
class DualNumber:
    def __init__(self, real, dual):
        """
        Initialize a dual number: real + ε * dual
        """
        self.real = real  # Function value
        self.dual = dual  # Derivative

    def __repr__(self):
        return f"DualNumber(real={self.real}, dual={self.dual})"

    def __add__(self, other):
        """
        Addition: (a + εb) + (c + εd) = (a + c) + ε(b + d)
          x = a + εb with b being the derivative of a
          y = c + εd with d being the derivative of c
          x + y = (a + c) + ε(b + d)
          Thus b+d is the derivative of a+c
        """
        other = other if isinstance(other, DualNumber) else DualNumber(other, 0)
        return DualNumber(self.real + other.real, self.dual + other.dual)

    def __mul__(self, other):
        """
        Multiplication: (a + εb) * (c + εd) = ac + ε(ad + bc)
          x = a + εb with b being the derivative of a
          y = c + εd with d being the derivative of c
          x * y = (a * c) + ε((a * d) + (b * c))
          Thus (a * d) + (b * c) is the derivative of a * c
        """
        other = other if isinstance(other, DualNumber) else DualNumber(other, 0)
        real_part = self.real * other.real
        dual_part = self.real * other.dual + self.dual * other.real
        return DualNumber(real_part, dual_part)

    def __pow__(self, exponent):
        """
        Power function: (x + εy)^n = x^n + ε(n * x^(n-1) * y)
          x = a + εb with b being the derivative of a
          n = exponent
          x^n = (a^n) + ε(n * a^(n-1) * b)
        """
        real_part = self.real ** exponent
        dual_part = exponent * (self.real ** (exponent - 1)) * self.dual
        return DualNumber(real_part, dual_part)


### **How Does This Work?**
- **Addition:** The derivative of $ f(x) + g(x) $ is just $ f'(x) + g'(x) $.
- **Multiplication:** The product rule states $ (fg)' = f'g + fg' $.
- **Power:** The power rule states $ (x^n)' = n x^{n-1} $.


## **3️⃣ Performing Automatic Differentiation with Dual Numbers**

### **Example 1: Compute the derivative of $ f(x) = x^2 + 3x + 5 $ at $ x=2 $**

$$
f(x) = x^2 + 3x + 5
$$

The derivative is:

$$
f'(x) = 2x + 3
$$

At $ x=2 $, we expect:

$$
f'(2) = 2(2) + 3 = 7
$$


In [2]:
# Define x as a dual number: real=2, dual=1 (since we want df/dx at x=2)
x = DualNumber(2, 1)

# Compute f(x) = x^2 + 3x + 5
f_x = (x**2) + (x * 3) + 5

# Print results
print(f"Function value at x=2: {f_x.real}")
print(f"Derivative at x=2: {f_x.dual}")


Function value at x=2: 15
Derivative at x=2: 7


## **🎯 Key Takeaways**
1. **Dual numbers encode both function values and derivatives.**
2. **Basic arithmetic rules allow differentiation to be computed automatically.**
3. **This method is efficient and forms the foundation of "forward-mode" autodiff.** as it propagates derivatives **forward** through the computation graph, evaluating derivatives alongside function values.


Forward-mode autodiff is inefficient for functions with many inputs and one output because it computes one derivative per function evaluation as shown below:

In [3]:
# Define the function f(x1, x2, x3, x4) = x1 * x2 + x3 * x4
def f(x1, x2, x3, x4):
    return (x1 * x2) + (x3 * x4)

# Input values
x1, x2, x3, x4 = 2, 3, 4, 5

# Compute gradients using forward-mode (one pass per variable)
grad_x1 = f(DualNumber(x1, 1), DualNumber(x2, 0), DualNumber(x3, 0), DualNumber(x4, 0)).dual
grad_x2 = f(DualNumber(x1, 0), DualNumber(x2, 1), DualNumber(x3, 0), DualNumber(x4, 0)).dual
grad_x3 = f(DualNumber(x1, 0), DualNumber(x2, 0), DualNumber(x3, 1), DualNumber(x4, 0)).dual
grad_x4 = f(DualNumber(x1, 0), DualNumber(x2, 0), DualNumber(x3, 0), DualNumber(x4, 1)).dual

# Print gradients
print(f"∂f/∂x1: {grad_x1}")
print(f"∂f/∂x2: {grad_x2}")
print(f"∂f/∂x3: {grad_x3}")
print(f"∂f/∂x4: {grad_x4}")


∂f/∂x1: 3
∂f/∂x2: 2
∂f/∂x3: 5
∂f/∂x4: 4


As you can see, this is quite inefficient as we had to have 4 foward passes to compute the derivatives with respect to the 4 inputs to the function.
In deep learning, computing the gradient of a loss function with respect to millions of parameters would require millions of forward-mode passes making forward mode differentiation, despite its mathematical beauty, computationally infeasible!

For this, we use backward mode differentiation.